In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
sns.set_palette("Set2")

In [2]:
from dotenv import load_dotenv
from ephys_queries import db_setup_core, select_spike_times, select_discrete_data
from spiketimes.df.alignment import align_around_by
from spiketimes.df.binning import binned_spiketrain_bins_provided
from spiketimes.df.baseline import zscore_standardise_by
from spiketimes.df.statistics import diffmeans_test_by
from spiketimes.utils import p_adjust
from spiketimes.plots import add_event_vlines

In [3]:
from scipy.ndimage import gaussian_filter1d
from scipy.stats import zscore
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score

In [4]:
data_dir = Path(".").absolute().parent / "data"
fig_dir = Path(".").absolute().parent / "figs"

In [5]:
group_names = ["acute_citalopram", 
            "acute_saline", 
            "shock", 
            "sham", 
            "acute_cit", 
            "acute_sal"]
dfb = (
    pd.read_csv(data_dir / "baseline.csv")
    .loc[lambda x: x["group_name"].isin(group_names)]
)

In [6]:
block_name = "base_shock"
fs = 30000
load_dotenv()
engine, metadata = db_setup_core()

df_spikes = (
    select_spike_times(
            engine, 
            metadata,
            group_names=group_names,
            block_name=block_name,
            )
    .assign(spiketimes=lambda x: x["spike_time_samples"].divide(fs))
)
df_events = (
    select_discrete_data(
            engine,
            metadata,
            group_names=group_names,
            block_name=block_name
    )
    .assign(event_s= lambda x: x["timepoint_sample"].divide(fs))
)

In [7]:
df_spikes = df_spikes.groupby("neuron_id").filter(lambda x: len(x) >= 1000)

In [8]:
bins = np.round(np.arange(-0.5, 1.51, 0.01), 3)

psth = (
    align_around_by(
        df_data=df_spikes.merge(dfb[["neuron_id", "session_name"]]).dropna(),
        df_data_data_colname="spiketimes",
        df_data_group_colname="session_name",
        df_events=df_events,
        df_events_event_colname="event_s",
        df_events_group_colname="session_name",
        max_latency=1.5,
        t_before=0.5)
    .dropna()
    .pipe(lambda x: binned_spiketrain_bins_provided(x, 
                                bins=bins, 
                                spiketimes_col="aligned", 
                                spiketrain_col="neuron_id")
         )
)

c:\users\rory\repos\spiketimes\spiketimes\alignment.py:140: RuntimeWarning: invalid value encountered in greater_equal
  (negative_latencies >= (t_before * -1)),
c:\users\rory\repos\spiketimes\spiketimes\alignment.py:148: RuntimeWarning: invalid value encountered in greater
  latencies[latencies > max_latency] = np.nan


In [9]:
psth.head(3)

,neuron_id,bin,counts
0,1031,-0.50,23
1,1031,-0.49,19
2,1031,-0.48,13


In [10]:
psth = (
    psth
    .assign(cond=(
        lambda x: x["bin"].apply(lambda y: 0 if (y < 0.01) or (y > 0.3) else 1)
            ))
)